In [3]:
import os
import pandas as pd
from consent import utils, ConSent, Config
from consent.model_selection import Handler

import json
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score
from sklearn.metrics import classification_report, multilabel_confusion_matrix

In [4]:
import dotenv
dotenv.load_dotenv(".env", override=True)

DATASET_NAME = os.environ['CONSENT_DATASET_NAME']
TRAIN_DATA_PATH = os.environ['CONSENT_TRAIN_DATA_PATH']
TEST_DATA_PATH = os.environ['CONSENT_TEST_DATA_PATH']
FINETUNING_DATASET_NAME = os.environ['CONSENT_FINETUNING_DATASET_NAME']
FINETUNING_TRAIN_DATA_PATH = os.environ['CONSENT_FINETUNING_TRAIN_DATA_PATH']
FINETUNING_TEST_DATA_PATH = os.environ['CONSENT_FINETUNING_TEST_DATA_PATH']
CONFIG_FOLDER = os.environ['CONSENT_CONFIG_FOLDER']
OUTPUTS_FOLDER = os.environ['CONSENT_OUTPUTS_FOLDER']
print(DATASET_NAME)
print(FINETUNING_DATASET_NAME)
print(CONFIG_FOLDER)
print(OUTPUTS_FOLDER)

# By convention from the dataset name
PRIMARY_LANGUAGE = DATASET_NAME.split('Chats-')[1][:2]
SECONDARY_LANGUAGE = FINETUNING_DATASET_NAME.split('Chats-')[1][:2]

Chats-NL-TW1ST_circuits
Chats-PT-Radiation
./configs/
./outputs/


In [5]:
code_name = 'L1'

In [4]:
# 1. Train a model from scratch
run = Handler(
    data_file_path = TRAIN_DATA_PATH,
    config_file_path = os.path.join(CONFIG_FOLDER,
        f"code_{code_name}__train__{PRIMARY_LANGUAGE}.json"),
    random_state = 1
)
run.train(save_model = os.path.join(OUTPUTS_FOLDER,
    f"models/code_{code_name}__{PRIMARY_LANGUAGE}"))

{
    "architecture": "consent==0.1.4",
    "caller": "train",
    "datasets": [],
    "dataset_name": "Chats-NL-TW1ST_circuits",
    "code_name": "L1",
    "codes": [
        "OFF",
        "COO",
        "DOM"
    ],
    "default_code": "OFF",
    "wandb_project": "consent-dev",
    "load_weights": null,
    "language_featurizer": "https://tfhub.dev/google/LEALLA/LEALLA-base/1",
    "sent_hl_units": 256,
    "sent_dropout": 0.2,
    "consent_hl_units": 32,
    "lags": 7,
    "augmentation": false,
    "n_samples": null,
    "max_epochs": 250,
    "callback_patience": 30,
    "learning_rate": 0.001,
    "batch_size": 256
}


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: adaj (use `wandb login --relogin` to force relogin)


Epoch 1/250
68/68 [==============================] 2.3099 - sent_output_loss: 1.0991 - consent_output_loss: 1.2108 - sent_output_accuracy: 0.3633 - sent_output_kappa: 0.0675 - sent_output_f1_score: 0.3633 - consent_output_accuracy: 0.2969 - consent_output_kappa: -0.0080 - consent_output_f1_score: 0.29 - 24s 6s/step - loss: 2.3047 - sent_output_loss: 1.0919 - consent_output_loss: 1.2128 - sent_output_accuracy: 0.3926 - sent_output_kappa: 0.0946 - sent_output_f1_score: 0.3926 - consent_output_accuracy: 0.3047 - consent_output_kappa: -0.0239 - consent_output_f1_score: 0.3047 - 30s 6s/step - loss: 2.2898 - sent_output_loss: 1.0905 - consent_output_loss: 1.1993 - sent_output_accuracy: 0.3854 - sent_output_kappa: 0.0822 - sent_output_f1_score: 0.3854 - consent_output_accuracy: 0.3112 - consent_output_kappa: -0.0111 - consent_output_f1_score: 0.311 - 36s 6s/step - loss: 2.2661 - sent_output_loss: 1.0845 - consent_output_loss: 1.1816 - sent_output_accuracy: 0.4189 - sent_output_kappa: 0.1229 -

INFO:tensorflow:Assets written to: ./outputs/models/code_L1__NL\assets


consent_output_accuracy,▁▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████████
consent_output_f1_score,▁▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████████
consent_output_kappa,▁▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████████
consent_output_loss,█▆▅▅▅▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▆▅▅▅▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
sent_output_accuracy,▁▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇███████████
sent_output_f1_score,▁▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇███████████
sent_output_kappa,▁▃▃▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇███████████
sent_output_loss,█▆▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
val_consent_output_accuracy,▁▃▄▅▆▆▇▇▇▇▇███████████████▇█▇▇█▇▇▇▇▇▇▇▇▇


In [5]:
# 2. Evaluate on test set (NL)
run = Handler(
    data_file_path = TEST_DATA_PATH,
    config_file_path = os.path.join(CONFIG_FOLDER,
        f"code_{code_name}__train__{PRIMARY_LANGUAGE}.json"),
    load_model = os.path.join(OUTPUTS_FOLDER,
        f"models/code_{code_name}__{PRIMARY_LANGUAGE}"),
    random_state = 1
)
run.test(
    dataset_name = DATASET_NAME,
    output_file_path = os.path.join(OUTPUTS_FOLDER,
        f"code_{code_name}__test__{PRIMARY_LANGUAGE}_{PRIMARY_LANGUAGE}.csv")
)

accuracy,▁
kappa,▁
micro-f1,▁
accuracy,0.7956
config,"{""architecture"": ""co..."
kappa,0.69147
micro-f1,0.7956


In [7]:
# 2. Evaluate on test set (PT)
run = Handler(
    data_file_path = FINETUNING_TEST_DATA_PATH,
    config_file_path = os.path.join(CONFIG_FOLDER,
        f"code_{code_name}__train__{PRIMARY_LANGUAGE}.json"),
    load_model = os.path.join(OUTPUTS_FOLDER,
        f"models/code_{code_name}__{PRIMARY_LANGUAGE}"),
    random_state = 1
)
run.test(
    dataset_name = FINETUNING_DATASET_NAME,
    output_file_path = os.path.join(OUTPUTS_FOLDER,
        f"code_{code_name}__test__{PRIMARY_LANGUAGE}_{SECONDARY_LANGUAGE}.csv")
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: adaj (use `wandb login --relogin` to force relogin)


accuracy,▁
kappa,▁
micro-f1,▁
accuracy,0.68182
config,"{""architecture"": ""co..."
kappa,0.52419
micro-f1,0.68182


In [6]:
# For L1 models, finetuning works better (see paper)

# 3. Fine tune with the Chats-PT-Radiation training set 
run = Handler(
    data_file_path = FINETUNING_TRAIN_DATA_PATH,
    config_file_path = os.path.join(CONFIG_FOLDER,
        f"code_{code_name}__fine_tune__{PRIMARY_LANGUAGE}_{SECONDARY_LANGUAGE}.json"),
    load_model = os.path.join(OUTPUTS_FOLDER,
        f"models/code_{code_name}__{PRIMARY_LANGUAGE}"),
    random_state = 1
)
run.fine_tune(
    save_model = os.path.join(OUTPUTS_FOLDER,
        f"models/code_{code_name}__fine_tuned__{PRIMARY_LANGUAGE}_{SECONDARY_LANGUAGE}")
)

{
    "architecture": "consent==0.1.4",
    "caller": "fine_tune",
    "datasets": [
        "Chats-NL-TW1ST_circuits+Chats-PT-Radiation"
    ],
    "dataset_name": "Chats-NL-TW1ST_circuits+Chats-PT-Radiation",
    "code_name": "L1",
    "codes": [
        "OFF",
        "COO",
        "DOM"
    ],
    "default_code": "OFF",
    "wandb_project": "consent-dev",
    "load_weights": "./outputs/models/code_L1__NL",
    "language_featurizer": "https://tfhub.dev/google/LEALLA/LEALLA-base/1",
    "sent_hl_units": 256,
    "sent_dropout": 0.2,
    "consent_hl_units": 32,
    "lags": 7,
    "augmentation": false,
    "n_samples": null,
    "max_epochs": 250,
    "callback_patience": 10,
    "learning_rate": 0.0001,
    "batch_size": 256
}


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: adaj (use `wandb login --relogin` to force relogin)


Epoch 1/250
7/7 [==============================]s: 1.7426 - sent_output_loss: 1.0460 - consent_output_loss: 0.6966 - sent_output_accuracy: 0.4961 - sent_output_kappa: 0.2595 - sent_output_f1_score: 0.4961 - consent_output_accuracy: 0.8008 - consent_output_kappa: 0.6910 - consent_output_f1_score: 0.800 - 30s 8s/step - loss: 1.6952 - sent_output_loss: 1.0330 - consent_output_loss: 0.6623 - sent_output_accuracy: 0.5234 - sent_output_kappa: 0.2956 - sent_output_f1_score: 0.5234 - consent_output_accuracy: 0.8262 - consent_output_kappa: 0.7348 - consent_output_f1_score: 0.826 - 38s 8s/step - loss: 1.7116 - sent_output_loss: 1.0311 - consent_output_loss: 0.6805 - sent_output_accuracy: 0.5286 - sent_output_kappa: 0.3032 - sent_output_f1_score: 0.5286 - consent_output_accuracy: 0.8034 - consent_output_kappa: 0.7017 - consent_output_f1_score: 0.80 - 45s 8s/step - loss: 1.7060 - sent_output_loss: 1.0287 - consent_output_loss: 0.6773 - sent_output_accuracy: 0.5371 - sent_output_kappa: 0.3146 - sen

INFO:tensorflow:Assets written to: ./outputs/models/code_L1__fine_tuned__NL_PT\assets


consent_output_accuracy,▁▂▂▃▃▄▄▅▅▄▅▅▅▅▆▆▆▆▆▅▆▆▇▆▇▇▇▇▇▇▇▇██▇█████
consent_output_f1_score,▁▂▂▃▃▄▄▅▅▄▅▅▅▅▆▆▆▆▆▅▆▆▇▆▇▇▇▇▇▇▇▇██▇█████
consent_output_kappa,▁▂▂▃▃▄▄▅▅▄▅▅▅▅▆▆▆▆▆▅▆▆▇▆▇▇▇▇▇▇▇▇▇█▇█████
consent_output_loss,█▇▇▆▅▅▅▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▆▆▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
sent_output_accuracy,▁▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇████████
sent_output_f1_score,▁▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇████████
sent_output_kappa,▁▂▃▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▅▆▆▆▆▆▇▇▇▇▇▇▇████████
sent_output_loss,█▇▇▆▆▆▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
val_consent_output_accuracy,▁▁▁▂▃▃▅▅▅▆▆▆▆▆▇▇▇████▇▆▆▆▆▆▆▇▇▇▇▇▆▆▇█▇█▇


In [7]:
# 4. Evaluate on test set (NL)
run = Handler(
    data_file_path = TEST_DATA_PATH,
    config_file_path = os.path.join(CONFIG_FOLDER,
        f"code_{code_name}__fine_tune__{PRIMARY_LANGUAGE}_{SECONDARY_LANGUAGE}.json"),
    load_model = os.path.join(OUTPUTS_FOLDER,
        f"models/code_{code_name}__fine_tuned__{PRIMARY_LANGUAGE}_{SECONDARY_LANGUAGE}"),
    random_state = 1
)
run.test(
    dataset_name = DATASET_NAME,
    output_file_path = os.path.join(OUTPUTS_FOLDER,
        f"code_{code_name}__test__{PRIMARY_LANGUAGE}_{PRIMARY_LANGUAGE}.csv")
)

accuracy,▁
kappa,▁
micro-f1,▁
accuracy,0.77405
config,"{""architecture"": ""co..."
kappa,0.65388
micro-f1,0.77405


In [8]:
# 4. Evaluate on test set (PT)
run = Handler(
    data_file_path = FINETUNING_TEST_DATA_PATH,
    config_file_path = os.path.join(CONFIG_FOLDER,
        f"code_{code_name}__fine_tune__{PRIMARY_LANGUAGE}_{SECONDARY_LANGUAGE}.json"),
    load_model = os.path.join(OUTPUTS_FOLDER,
        f"models/code_{code_name}__fine_tuned__{PRIMARY_LANGUAGE}_{SECONDARY_LANGUAGE}"),
    random_state = 1
)
run.test(
    dataset_name = FINETUNING_DATASET_NAME,
    output_file_path = os.path.join(OUTPUTS_FOLDER,
        f"code_{code_name}__test__{PRIMARY_LANGUAGE}_{SECONDARY_LANGUAGE}.csv")
)

accuracy,▁
kappa,▁
micro-f1,▁
accuracy,0.85606
config,"{""architecture"": ""co..."
kappa,0.78257
micro-f1,0.85606


## L2C

In [9]:
code_name = 'L2C'

In [11]:
# For L2C models, new training works better (see paper)

# Train from scratch with both (train) datasets together
run = Handler(
    data_file_path = [
        TRAIN_DATA_PATH,
        FINETUNING_TRAIN_DATA_PATH,
    ],
    config_file_path = os.path.join(CONFIG_FOLDER,
        f"code_{code_name}__train__{PRIMARY_LANGUAGE}+{SECONDARY_LANGUAGE}.json"),
    random_state = 1
)
run.train(save_model = os.path.join(OUTPUTS_FOLDER,
    f"models/code_{code_name}__{PRIMARY_LANGUAGE}+{SECONDARY_LANGUAGE}"))

# model_l2c = run.model

{
    "architecture": "consent==0.1.4",
    "caller": "train",
    "datasets": [],
    "dataset_name": "Chats-NL-TW1ST_circuits+Chats-PT-Radiation",
    "code_name": "L2C",
    "codes": [
        "IN",
        "AR",
        "AI",
        "AM",
        "NOS"
    ],
    "default_code": "NOS",
    "wandb_project": "consent-dev",
    "load_weights": null,
    "language_featurizer": "https://tfhub.dev/google/LEALLA/LEALLA-base/1",
    "sent_hl_units": 512,
    "sent_dropout": 0.4,
    "consent_hl_units": 32,
    "lags": 7,
    "augmentation": false,
    "n_samples": null,
    "max_epochs": 250,
    "callback_patience": 30,
    "learning_rate": 0.0005,
    "batch_size": 256
}


Epoch 1/250
75/75 [==============================] 3.3271 - sent_output_loss: 1.6168 - consent_output_loss: 1.7102 - sent_output_accuracy: 0.1875 - sent_output_kappa: -0.0051 - sent_output_f1_score: 0.1875 - consent_output_accuracy: 0.1328 - consent_output_kappa: 0.0022 - consent_output_f1_score: 0.13 - 23s 6s/step - loss: 3.3268 - sent_output_loss: 1.6088 - consent_output_loss: 1.7179 - sent_output_accuracy: 0.2246 - sent_output_kappa: -0.0231 - sent_output_f1_score: 0.2246 - consent_output_accuracy: 0.1309 - consent_output_kappa: -0.0136 - consent_output_f1_score: 0.13 - 30s 6s/step - loss: 3.3234 - sent_output_loss: 1.6007 - consent_output_loss: 1.7227 - sent_output_accuracy: 0.2695 - sent_output_kappa: -0.0056 - sent_output_f1_score: 0.2695 - consent_output_accuracy: 0.1432 - consent_output_kappa: -0.0013 - consent_output_f1_score: 0.14 - 36s 6s/step - loss: 3.3061 - sent_output_loss: 1.5921 - consent_output_loss: 1.7140 - sent_output_accuracy: 0.3008 - sent_output_kappa: -0.0051 -

INFO:tensorflow:Assets written to: ./outputs/models/code_L2C__NL+PT\assets


consent_output_accuracy,▁▄▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████████████
consent_output_f1_score,▁▄▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████████████
consent_output_kappa,▁▂▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████████
consent_output_loss,█▆▅▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▅▅▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
sent_output_accuracy,▁▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇███████████
sent_output_f1_score,▁▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇███████████
sent_output_kappa,▁▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████████
sent_output_loss,█▅▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
val_consent_output_accuracy,▁▄▆▇▇▇▇▇▇▇▇█████████████████████████████


In [15]:
# 4. Evaluate on test set (NL)
run = Handler(
    data_file_path = TEST_DATA_PATH,
    config_file_path = os.path.join(CONFIG_FOLDER,
        f"code_{code_name}__train__{PRIMARY_LANGUAGE}+{SECONDARY_LANGUAGE}.json"),
    load_model = os.path.join(OUTPUTS_FOLDER,
        f"models/code_{code_name}__{PRIMARY_LANGUAGE}+{SECONDARY_LANGUAGE}"),
    random_state = 1
)
run.test(
    dataset_name = DATASET_NAME,
    output_file_path = os.path.join(OUTPUTS_FOLDER,
        f"code_{code_name}__test__{PRIMARY_LANGUAGE}_{PRIMARY_LANGUAGE}.csv")
)

accuracy,▁
kappa,▁
micro-f1,▁
accuracy,0.78076
config,"{""architecture"": ""co..."
kappa,0.66201
micro-f1,0.78076


In [16]:
# 4. Evaluate on test set (PT)
run = Handler(
    data_file_path = FINETUNING_TEST_DATA_PATH,
    config_file_path = os.path.join(CONFIG_FOLDER,
        f"code_{code_name}__train__{PRIMARY_LANGUAGE}+{SECONDARY_LANGUAGE}.json"),
    load_model = os.path.join(OUTPUTS_FOLDER,
        f"models/code_{code_name}__{PRIMARY_LANGUAGE}+{SECONDARY_LANGUAGE}"),
    random_state = 1
)
run.test(
    dataset_name = FINETUNING_DATASET_NAME,
    output_file_path = os.path.join(OUTPUTS_FOLDER,
        f"code_{code_name}__test__{PRIMARY_LANGUAGE}_{PRIMARY_LANGUAGE}.csv")
)

accuracy,▁
kappa,▁
micro-f1,▁
accuracy,0.71212
config,"{""architecture"": ""co..."
kappa,0.58643
micro-f1,0.71212
